In [1]:
import pandas as pd
from nltk.util import ngrams
import numpy as np
import os 
import unicodedata
import nltk

In [2]:
df  =  pd.read_excel('name_data/exigerData/EXGR_Korean names.xlsx')
print(df)

       Unnamed: 0                                        id       fullname  \
0               0    http://www.wikidata.org/entity/Q484396  Park Joo-bong   
1               1  http://www.wikidata.org/entity/Q55728351  KIM Jong hoon   
2               2  http://www.wikidata.org/entity/Q11266766            이민혁   
3               3  http://www.wikidata.org/entity/Q47492159         Lee Ho   
4               4   http://www.wikidata.org/entity/Q1075756            최민호   
...           ...                                       ...            ...   
21197       21197  http://www.wikidata.org/entity/Q11267007     Lee Han-wi   
21198       21198  http://www.wikidata.org/entity/Q12586843   Gil Jung-woo   
21199       21199    http://www.wikidata.org/entity/Q224639            이정희   
21200       21200  http://www.wikidata.org/entity/Q12586585            금태섭   
21201       21201  http://www.wikidata.org/entity/Q18684836      Oh Ui-sik   

       Family name  Given name  
0              NaN         NaN

In [13]:
#checking the shape(21202, 5)
df.shape

(21202, 5)

In [14]:
#checking the head 
df.head()

,Unnamed: 0,id,fullname,Family name,Given name
0,0,http://www.wikidata.org/entity/Q484396,Park Joo-bong,NaN,NaN
1,1,http://www.wikidata.org/entity/Q55728351,KIM Jong hoon,NaN,NaN
2,2,http://www.wikidata.org/entity/Q11266766,이민혁,NaN,NaN
3,3,http://www.wikidata.org/entity/Q47492159,Lee Ho,NaN,NaN
4,4,http://www.wikidata.org/entity/Q1075756,최민호,NaN,NaN


In [15]:
#checking how many null entries each columns have( 21202 for family and given name)
nan_count = np.sum(df.isnull(), axis = 0)
nan_count

Unnamed: 0         0
id                 0
fullname           0
Family name    21202
Given name     21202
dtype: int64

In [16]:
#checking if there's any duplicates(yes)
df['fullname'].duplicated().any()

True

In [17]:
#drop them from the fullname  and checking if nullentries went down(form 21202 to 19520)
df2 =df.drop_duplicates(subset=['fullname'])

nan_count = np.sum(df2.isnull(), axis = 0)
nan_count

Unnamed: 0         0
id                 0
fullname           0
Family name    19520
Given name     19520
dtype: int64

In [19]:
#now the duplicates are gone
df2['fullname'].duplicated().any()

False

Now For Feature Engineering 

In [22]:
#determining the lan
def Korean_lan(name):
    if isinstance(name, str):
        return [unicodedata.name(char).split(' ')[0] for char in name]
    else:
        return None

# Apply the function only to rows where 'fullname' is a string
df2['determine_alphabet'] = df2['fullname'].apply(Korean_lan)
print(df2['determine_alphabet'])

0        [LATIN, LATIN, LATIN, LATIN, SPACE, LATIN, LAT...
1        [LATIN, LATIN, LATIN, SPACE, LATIN, LATIN, LAT...
2                                 [HANGUL, HANGUL, HANGUL]
3               [LATIN, LATIN, LATIN, SPACE, LATIN, LATIN]
4                                 [HANGUL, HANGUL, HANGUL]
                               ...                        
21197    [LATIN, LATIN, LATIN, SPACE, LATIN, LATIN, LAT...
21198    [LATIN, LATIN, LATIN, SPACE, LATIN, LATIN, LAT...
21199                             [HANGUL, HANGUL, HANGUL]
21200                             [HANGUL, HANGUL, HANGUL]
21201    [LATIN, LATIN, SPACE, LATIN, LATIN, HYPHEN-MIN...
Name: determine_alphabet, Length: 19520, dtype: object


/var/folders/4g/6d3mv01d7k7dpqlb46ckdk_00000gn/T/ipykernel_63757/1593449575.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['determine_alphabet'] = df2['fullname'].apply(Korean_lan)


In [27]:
#featuring the char_ngrams (Jolie way)
"""
my way don't work correctly 
def korean_char_ngrams(names,n):
    if not isinstance(names,str):
        return []
    
    char_ngrams = [names[i:i+n] for i in range(len(names)-n +1)]
    return char_ngrams

n = 3
df2['char_ngrams'] = df2['fullname'].apply(lambda x: korean_char_ngrams(x, n))
"""
def korean_char_ngrams(names,n):
    if not isinstance(names,str):
        return []
    
    words = text.split()
    
    unigrams = list(ngrams(text,1))
    bigrams = list(ngrams(text, 2))
    trigrams = list(ngrams(text, 3))
    

print(df2['char_ngrams'])

0        [Par, ark, rk , k J,  Jo, Joo, oo-, o-b, -bo, ...
1        [KIM, IM , M J,  Jo, Jon, ong, ng , g h,  ho, ...
2                                                    [이민혁]
3                                     [Lee, ee , e H,  Ho]
4                                                    [최민호]
                               ...                        
21197             [Lee, ee , e H,  Ha, Han, an-, n-w, -wi]
21198    [Gil, il , l J,  Ju, Jun, ung, ng-, g-w, -wo, ...
21199                                                [이정희]
21200                                                [금태섭]
21201                  [Oh , h U,  Ui, Ui-, i-s, -si, sik]
Name: char_ngrams, Length: 19520, dtype: object


In [26]:
#feature for token length 
def token_length(name):
    if isinstance(name,str):
       return len(name.split())
    else:
       return None

df2['token_len'] = df2['fullname'].apply(token_length)
print(df2['token_len'])

0        2.0
1        3.0
2        1.0
3        2.0
4        1.0
        ... 
21197    2.0
21198    2.0
21199    1.0
21200    1.0
21201    2.0
Name: token_len, Length: 19520, dtype: float64


/var/folders/4g/6d3mv01d7k7dpqlb46ckdk_00000gn/T/ipykernel_63757/4027173506.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['token_len'] = df2['fullname'].apply(token_length)
